In [30]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint

def create_datagen():
    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  
    )
    return train_datagen

def create_generators(base_dir, input_shape, batch_size):
    datagen = create_datagen()
    
    train_generator = datagen.flow_from_directory(
        base_dir,  
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        base_dir,  
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator



def define_model(num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(34, 80, 3)),  
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'),
        Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(100, activation='relu', kernel_initializer='he_uniform'),
        Dense(num_classes, activation='softmax')
    ])
    
    opt = SGD(learning_rate=0.001, momentum=0.01)  
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model




base_dir = 'supervisedLetters'
input_shape = (34, 80)  
batch_size = 16
num_classes = 36  


train_gen, val_gen = create_generators(base_dir, input_shape, batch_size)
print(train_gen.class_indices)


model = define_model(num_classes)


model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)  


checkpoint_cb = ModelCheckpoint(
    os.path.join(model_dir, 'model_epoch_{epoch:02d}_loss_{val_loss:.8f}.keras'),
    save_best_only=False,  
    monitor='val_loss',  
    mode='min',  
    verbose=1  
)


model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    validation_data=val_gen,
    validation_steps=len(val_gen),
    epochs=1000,
    callbacks=[checkpoint_cb]
)


Found 6085 images belonging to 36 classes.
Found 1503 images belonging to 36 classes.
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33, 'y': 34, 'z': 35}
Epoch 1/1000
378/381 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0351 - loss: 3.6237
Epoch 1: saving model to models\model_epoch_01_loss_3.58627963.keras
381/381 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.0351 - loss: 3.6234 - val_accuracy: 0.0259 - val_loss: 3.5863
Epoch 2/1000

Epoch 2: saving model to models\model_epoch_02_loss_0.00000000.keras
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 211us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/1000
380/381 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0292 - loss: 3.5817
Epoch 3: saving